Import packages and define functions

In [1]:
import numpy as np
import datetime
import pandas as pd
import xarray as xa

from scipy.stats.stats import pearsonr
from netCDF4 import Dataset
from glob import glob

In [2]:
def to_decimal_year(time):
    dt = pd.to_datetime(time) # converts np.datetime64 into datetime object
    dec_year = []
    for i in range(len(dt)):
        start = datetime.date(dt[i].year,1,1).toordinal()
        year_length = datetime.date(dt[i].year+1, 1, 1).toordinal() - start
        dec_year.append(dt[i].year + (float(dt[i].toordinal() - start) / year_length)) # calculates how much time has passed in the year, divides by year length, then adds to start year
    return np.array(dec_year)

In [3]:
def to_decimal_year(time):
    dt = pd.to_datetime(time) # converts np.datetime64 into datetime object
    dec_year = []
    for i in range(len(dt)):
        start = datetime.date(dt[i].year,1,1).toordinal()
        year_length = datetime.date(dt[i].year+1, 1, 1).toordinal() - start
        dec_year.append(dt[i].year + (float(dt[i].toordinal() - start) / year_length)) # calculates how much time has passed in the year, divides by year length, then adds to start year
    return np.array(dec_year)
def lag_corr(array1,array2,lag):
    if lag>0:
        return pearsonr(array2[lag:-1],array1[:-1-lag])
    elif lag<0:
        l = int(-1.*lag)
        return pearsonr(array1[l:-1],array2[:-1-l])
    else:
        return pearsonr(array1,array2)
def detrend_from_coefs(time,values,coefs):
    yr = to_decimal_year(time)
    
    mean = coefs[0]
    trend = coefs[1]*(yr-np.mean(yr))
    res_coefs = coefs[2:]
    
    detrended = values.copy()
    detrended -= mean
    detrended -= trend
    for j in range(0,len(res_coefs),2):
        omega = (j+2)*np.pi
        detrended -= (res_coefs[j]*np.sin(omega*yr))+(res_coefs[j+1]*np.cos(omega*yr))
    
    return detrended  

Extract data

In [4]:
files = np.sort(glob('/Users/erickson/Documents/Data/RFROM/*.nc'))
data = xa.open_mfdataset(files)
time = data['time'].values

In [5]:
coefs = Dataset('trend.nc','r')

In [6]:
deps = np.array([2.5,100,150,200])
depinds = np.where(np.isin(data.variables['mean_pressure'].values,deps))[0]

In [7]:
lags = np.arange(-52*3,52*3+1)
lats = data['latitude'].values
lons = data['longitude'].values
lagged_corr = np.empty(shape=(len(deps),len(lats),len(lons),len(lags)))*np.nan
lagged_p = np.empty(shape=(len(deps),len(lats),len(lons),len(lags)))*np.nan

In [28]:
nc_out = Dataset('lagged_correlation_analysis.nc','w')
nc_out.createDimension('lag',len(lags))
nc_out.createDimension('dep',len(deps))
nc_out.createDimension('lat',len(lats))
nc_out.createDimension('lon',len(lons))
var = nc_out.createVariable('lag','i8',('lag'))
var.units = 'weeks'
var[:] = lags
var = nc_out.createVariable('dep','f4',('dep'))
var.units = 'dbar'
var[:] = deps
nc_out.createVariable('lat','f4',('lat'))[:] = lats
nc_out.createVariable('lon','f4',('lon'))[:] = lons
nc_out.createVariable('lagged_corr','f4',('lag','dep','lat','lon'))
nc_out.createVariable('lagged_p','f4',('lag','dep','lat','lon'))
nc_out.close()

In [9]:
for j in range(len(lats)):
    print(lats[j],end='')
    if lats[j]<-50:
        continue
    for k in range(len(lons)):
        print('.',end='')
        if np.isnan(coefs.variables['coefs'][0,0,j,k]):
            continue
        surf_data = detrend_from_coefs(time,data.variables['ocean_temperature'][:,0,j,k].values,coefs.variables['coefs'][:,0,j,k])
        if np.any(np.isnan(surf_data)) or np.any(np.isinf(surf_data)):
            continue
        for i,depind in enumerate(depinds):
            
            if i==0:
                dep_data = surf_data.copy()
            else:
                dep_data = detrend_from_coefs(time,data.variables['ocean_temperature'][:,depind,j,k].values,coefs.variables['coefs'][:,depind,j,k])
            if np.any(np.isnan(dep_data)) or np.any(np.isinf(dep_data)):
                continue
            for l in range(len(lags)):
                lag = lags[l]
                lagged_corr,lagged_p = lag_corr(surf_data,dep_data,lag);
                nc_out = Dataset('lagged_correlation_analysis.nc','a')
                nc_out.variables['lagged_corr'][l,i,j,k] = lagged_corr
                nc_out.variables['lagged_p'][l,i,j,k] = lagged_p
                nc_out.close()

-89.875-88.875-87.875-86.875-85.875-84.875-83.875-82.875-81.875-80.875-79.875-78.875-77.875-76.875-75.875-74.875-73.875-72.875-71.875-70.875-69.875-68.875-67.875-66.875-65.875-64.875-63.875-62.875-61.875-60.875-59.875-58.875-57.875-56.875-55.875-54.875-53.875-52.875-51.875-50.875-49.875........................................................................................................................................................................................................................................................................................................................................................................-48.875..........................................................................................................................................................................................................................................................................................................................................................

In [ ]:
coefs.close()